In [96]:
import pandas as pd
import numpy as np
import sys
import os
from datetime import datetime, time
sys.path.append(os.path.abspath(".."))
from core.viz import plot_line, create_subplot_grid, plot_bar, plot_statistical_strip, plot_scatter
from core.s3 import S3AssetManager

In [67]:
notebook_name = "exp_central_ganadera_tiempos"
s3 = S3AssetManager(notebook_name=notebook_name)

In [68]:
df = pd.read_excel("../raw/RG L 04 Informe Resultado Laboratorio 2025.xlsx",
sheet_name="TRAZABILIDAD PRODUCTO", skiprows=6)
df.columns = df.columns.str.strip().str.lower()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [69]:
df.columns

Index(['#', 'día muestreo', 'mes muestreo', 'año muestreo', 'fecha',
       'hora \ntoma de muestra', 'municipio', 'granja',
       'productor/cliente planta', 'lugar\ntoma de muestra', 'día\ning lab',
       'hora \ning lab', 'tipo de muestra\nproducto', 'id muestra',
       '#cava #muestra', 't°c/ frío', 'responsable muestreo', 'procesado por',
       'día \nsiembra / incubadora', 'hora incubación',
       'análisis realizado e. coli\nincubación a 36°c por 24 horas',
       'conteo placa ec', 'resultado ec ufc/cm2', 'm ec', 'm ec4',
       'interpretación ec',
       'análisis realizado coliformes totales\n incubación a 36°c por 24 horas',
       'conteo placa ct', 'resultado ct ufc/cm2', 'm ct', 'm ct5',
       'interpretación ct',
       'análisis realizado salmonella spp\nincubación a 41,5°c por 24 horas',
       'resultado salmonella', 'interpretación salmonella',
       'confirmación laboratorio externo salmonella',
       'análisis realizado e. coli o157:h7\nincubación a 41,5°c

In [70]:
rename_map = {
    "#": "id",
    "día muestreo": "dia_muestreo",
    "mes muestreo": "mes_muestreo",
    "año muestreo": "anio",
    "fecha": "fecha",
    "hora \ntoma de muestra": "hora_toma_muestra",
    "municipio": "municipio",
    "granja": "granja",
    "productor/cliente planta": "productor_cliente_planta",
    "lugar\ntoma de muestra": "lugar_toma_muestra",
    "día\ning lab": "dia_ingreso_lab",
    "hora \ning lab": "hora_ingreso_lab",
    "tipo de muestra\nproducto": "tipo_muestra_producto",
    "id muestra": "id_muestra",
    "#cava #muestra": "cava_muestra",
    "t°c/ frío": "temperatura_c_frio",
    "responsable muestreo": "responsable_muestreo",
    "procesado por": "procesado_por",
    "día \nsiembra / incubadora": "dia_siembra_incubadora",
    "hora incubación": "hora_incubacion",

    "análisis realizado e. coli\nincubación a 36°c por 24 horas": "analisis_ecoli_36c_24h",
    "conteo placa ec": "conteo_placa_ec",
    "resultado ec ufc/cm2": "resultado_ec_ufc_cm2",
    "m ec": "m_ec",
    "m ec4": "m_ec4",
    "interpretación ec": "interpretacion_ec",

    "análisis realizado coliformes totales\n incubación a 36°c por 24 horas": "analisis_coliformes_totales_36c_24h",
    "conteo placa ct": "conteo_placa_ct",
    "resultado ct ufc/cm2": "resultado_ct_ufc_cm2",
    "m ct": "m_ct",
    "m ct5": "m_ct5",
    "interpretación ct": "interpretacion_ct",

    "análisis realizado salmonella spp\nincubación a 41,5°c por 24 horas": "analisis_salmonella_spp_41_5c_24h",
    "resultado salmonella": "resultado_salmonella",
    "interpretación salmonella": "interpretacion_salmonella",
    "confirmación laboratorio externo salmonella": "confirmacion_lab_externo_salmonella",

    "análisis realizado e. coli o157:h7\nincubación a 41,5°c por 24 horas": "analisis_ecoli_o157_h7_41_5c_24h",
    "resultado ec o157:h7": "resultado_ecoli_o157_h7",
    "interpretación ec o157:h7": "interpretacion_ecoli_o157_h7",
    "confirmación laboratorio externo ec o157:h7": "confirmacion_lab_externo_ecoli_o157_h7",

    "obs": "observaciones",
    "dato erróneo que se corrigió (escriba el dato malo aquí)": "dato_erroneo_corregido",
    "responsable del error": "responsable_error",
    "fecha en la que se hace la corrección": "fecha_correccion",
    "responsable corrección": "responsable_correccion",
    "revisado por": "revisado_por",
    "fecha revisión": "fecha_revision",
}

df = df.rename(columns=rename_map)


In [71]:
df.columns

Index(['id', 'dia_muestreo', 'mes_muestreo', 'anio', 'fecha',
       'hora_toma_muestra', 'municipio', 'granja', 'productor_cliente_planta',
       'lugar_toma_muestra', 'dia_ingreso_lab', 'hora_ingreso_lab',
       'tipo_muestra_producto', 'id_muestra', 'cava_muestra',
       'temperatura_c_frio', 'responsable_muestreo', 'procesado_por',
       'dia_siembra_incubadora', 'hora_incubacion', 'analisis_ecoli_36c_24h',
       'conteo_placa_ec', 'resultado_ec_ufc_cm2', 'm_ec', 'm_ec4',
       'interpretacion_ec', 'analisis_coliformes_totales_36c_24h',
       'conteo_placa_ct', 'resultado_ct_ufc_cm2', 'm_ct', 'm_ct5',
       'interpretacion_ct', 'analisis_salmonella_spp_41_5c_24h',
       'resultado_salmonella', 'interpretacion_salmonella',
       'confirmacion_lab_externo_salmonella',
       'analisis_ecoli_o157_h7_41_5c_24h', 'resultado_ecoli_o157_h7',
       'interpretacion_ecoli_o157_h7',
       'confirmacion_lab_externo_ecoli_o157_h7', 'observaciones',
       'dato_erroneo_corregido', '

In [72]:
def assemble_dt(row, d_col, t_col):
    try:
        # Extraer año, mes y día (asegurando enteros)
        y = int(row['anio'])
        m = int(row['mes_muestreo'])
        d = int(row[d_col])
        
        # Manejar la hora
        t_val = row[t_col]
        
        # Si ya es un objeto time de Python
        if isinstance(t_val, time):
            h, mi, s = t_val.hour, t_val.minute, t_val.second
        else:
            # Si es un string o float, intentar partirlo
            parts = str(t_val).split(':')
            h = int(float(parts[0]))
            mi = int(float(parts[1])) if len(parts) > 1 else 0
            s = int(float(parts[2])) if len(parts) > 2 else 0
            
        return datetime(y, m, d, h, mi, s)
    except:
        return pd.NaT



In [ ]:

df['sample_date'] = df.apply(lambda r: assemble_dt(r, 'dia_muestreo', 'hora_toma_muestra'), axis=1)
df['fecha_ingreso'] = df.apply(lambda r: assemble_dt(r, 'dia_ingreso_lab', 'hora_ingreso_lab'), axis=1)
df['fecha_incubacion'] = df.apply(lambda r: assemble_dt(r, 'dia_siembra_incubadora', 'hora_incubacion'), axis=1)


df['horas_toma_a_ingreso'] = (df['fecha_ingreso'] - df['sample_date']).dt.total_seconds() / 3600
df['horas_ingreso_a_incubacion'] = (df['fecha_incubacion'] - df['fecha_ingreso']).dt.total_seconds() / 3600
df['horas_toma_a_incubacion'] = (df['fecha_incubacion'] - df['sample_date']).dt.total_seconds() / 3600

In [82]:
cond = (df['horas_toma_a_incubacion']<0) | (df['horas_ingreso_a_incubacion']<0) | (df['horas_toma_a_ingreso']<0)
df_good = df[~cond]

In [85]:
cls = [
'id', 
#'fecha', 
#'dia_muestreo',
#'mes_muestreo', 
#'anio',
#'hora_toma_muestra', 
#'dia_ingreso_lab', 
#'hora_ingreso_lab',
#'sample_date',
'horas_toma_a_ingreso', 
'horas_ingreso_a_incubacion',
'horas_toma_a_incubacion',
#'fecha_ingreso',
#'fecha_incubacion',
'tipo_muestra_producto', 
#'cava_muestra',
#'dia_siembra_incubadora',
#'hora_incubacion', 
'analisis_ecoli_36c_24h',
'resultado_ec_ufc_cm2',
'analisis_coliformes_totales_36c_24h',
'resultado_ct_ufc_cm2',
'analisis_salmonella_spp_41_5c_24h',
'resultado_salmonella',
'analisis_ecoli_o157_h7_41_5c_24h',
'resultado_ecoli_o157_h7',
]
df_good[cls]

,id,horas_toma_a_ingreso,horas_ingreso_a_incubacion,horas_toma_a_incubacion,tipo_muestra_producto,analisis_ecoli_36c_24h,resultado_ec_ufc_cm2,analisis_coliformes_totales_36c_24h,resultado_ct_ufc_cm2,analisis_salmonella_spp_41_5c_24h,resultado_salmonella,analisis_ecoli_o157_h7_41_5c_24h,resultado_ecoli_o157_h7
0,P00001,0.166667,10.000000,10.166667,Canal Porcina,Petrifilm EC: E. coli gen,1.50,Petrifilm EC: Col Tot,2.0,No realizado,NaN,No realizado,NaN
2,P00003,0.216667,106.700000,106.916667,Canal Bovina,No realizado,NaN,No realizado,NaN,No realizado,NaN,mini VIDAS E. coli O157:H7,0.0
4,P00005,0.216667,7.166667,7.383333,Canal Bovina,Petrifilm EC: E. coli gen,0.25,Petrifilm EC: Col Tot,0.0,No realizado,NaN,No realizado,NaN
5,P00006,0.250000,7.416667,7.666667,Canal Porcina,Petrifilm EC: E. coli gen,0.42,Petrifilm EC: Col Tot,1.0,No realizado,NaN,No realizado,NaN
6,P00007,0.166667,6.166667,6.333333,Canal Bovina,Petrifilm EC: E. coli gen,0.08,Petrifilm EC: Col Tot,0.0,No realizado,NaN,No realizado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8062,P08063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8063,P08064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8064,P08065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8065,P08066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [91]:
df_good["analisis_ecoli_36c_24h"].value_counts(dropna=False)

Petrifilm EC: E. coli gen    5858
No realizado                 1380
NaN                           450
Name: analisis_ecoli_36c_24h, dtype: int64

In [98]:
analisis_ecoli_36c_24h = df_good[df_good["analisis_ecoli_36c_24h"] == 'Petrifilm EC: E. coli gen']

analisis_ecoli_36c_24h[cls]

,id,horas_toma_a_ingreso,horas_ingreso_a_incubacion,horas_toma_a_incubacion,tipo_muestra_producto,analisis_ecoli_36c_24h,resultado_ec_ufc_cm2,analisis_coliformes_totales_36c_24h,resultado_ct_ufc_cm2,analisis_salmonella_spp_41_5c_24h,resultado_salmonella,analisis_ecoli_o157_h7_41_5c_24h,resultado_ecoli_o157_h7
0,P00001,0.166667,10.000000,10.166667,Canal Porcina,Petrifilm EC: E. coli gen,1.50,Petrifilm EC: Col Tot,2.0,No realizado,NaN,No realizado,NaN
4,P00005,0.216667,7.166667,7.383333,Canal Bovina,Petrifilm EC: E. coli gen,0.25,Petrifilm EC: Col Tot,0.0,No realizado,NaN,No realizado,NaN
5,P00006,0.250000,7.416667,7.666667,Canal Porcina,Petrifilm EC: E. coli gen,0.42,Petrifilm EC: Col Tot,1.0,No realizado,NaN,No realizado,NaN
6,P00007,0.166667,6.166667,6.333333,Canal Bovina,Petrifilm EC: E. coli gen,0.08,Petrifilm EC: Col Tot,0.0,No realizado,NaN,No realizado,NaN
7,P00008,0.216667,0.333333,0.550000,VR Porcina,Petrifilm EC: E. coli gen,1.50,Petrifilm EC: Col Tot,3.0,No realizado,NaN,No realizado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7612,P07613,NaN,NaN,NaN,NaN,Petrifilm EC: E. coli gen,NaN,Petrifilm EC: Col Tot,NaN,No realizado,NaN,No realizado,NaN
7613,P07614,NaN,NaN,NaN,NaN,Petrifilm EC: E. coli gen,NaN,Petrifilm EC: Col Tot,NaN,No realizado,NaN,No realizado,NaN
7614,P07615,NaN,NaN,NaN,NaN,Petrifilm EC: E. coli gen,NaN,Petrifilm EC: Col Tot,NaN,No realizado,NaN,No realizado,NaN
7615,P07616,NaN,NaN,NaN,NaN,Petrifilm EC: E. coli gen,NaN,Petrifilm EC: Col Tot,NaN,No realizado,NaN,No realizado,NaN


In [102]:
plot_scatter(
    df=analisis_ecoli_36c_24h[analisis_ecoli_36c_24h["resultado_ec_ufc_cm2"]>0],
     x_col="horas_ingreso_a_incubacion",
      y_col="resultado_ec_ufc_cm2",
       group_col="tipo_muestra_producto", 
       size_col="", 
       order_groups=None, 
       hover_data_cols=None, 
       title="", 
       x_title=None, 
       y_title=None, 
       text_format=".1f", 
       marker_size=10, 
       marker_opacity=0.8, 
       marker_colors=None, 
       height=600, 
       width=1000, 
       font_family="Inter, Arial, sans-serif", 
       output_path=None
)

In [104]:
plot_scatter(
    df=analisis_ecoli_36c_24h[analisis_ecoli_36c_24h["resultado_ec_ufc_cm2"]>0],
     x_col="horas_toma_a_ingreso",
      y_col="horas_toma_a_incubacion",
       group_col="tipo_muestra_producto", 
       size_col="", 
       order_groups=None, 
       hover_data_cols=None, 
       title="", 
       x_title=None, 
       y_title=None, 
       text_format=".1f", 
       marker_size=10, 
       marker_opacity=0.8, 
       marker_colors=None, 
       height=600, 
       width=1000, 
       font_family="Inter, Arial, sans-serif", 
       output_path=None
)

In [107]:
salmo = df_good[df_good["resultado_salmonella"].notnull()][cls]
salmo

,id,horas_toma_a_ingreso,horas_ingreso_a_incubacion,horas_toma_a_incubacion,tipo_muestra_producto,analisis_ecoli_36c_24h,resultado_ec_ufc_cm2,analisis_coliformes_totales_36c_24h,resultado_ct_ufc_cm2,analisis_salmonella_spp_41_5c_24h,resultado_salmonella,analisis_ecoli_o157_h7_41_5c_24h,resultado_ecoli_o157_h7
8,P00009,0.233333,104.233333,104.466667,Canal Porcina,No realizado,NaN,No realizado,NaN,mini VIDAS Salmonella spp,0.0,No realizado,NaN
11,P00012,0.200000,96.000000,96.200000,Canal Bovina >7 °c,No realizado,NaN,No realizado,NaN,mini VIDAS Salmonella spp,0.0,No realizado,NaN
13,P00014,0.250000,85.366667,85.616667,Canal Porcina,No realizado,NaN,No realizado,NaN,mini VIDAS Salmonella spp,0.0,No realizado,NaN
16,P00017,0.183333,82.700000,82.883333,Canal Bovina,No realizado,NaN,No realizado,NaN,mini VIDAS Salmonella spp,0.0,No realizado,NaN
23,P00024,0.250000,12.650000,12.900000,Carne Cabeza Bovina,Petrifilm EC: E. coli gen,260.0,Petrifilm EC: Col Tot,390.0,mini VIDAS Salmonella spp,0.0,No realizado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5035,P05036,0.250000,9.950000,10.200000,VB Bovina,Petrifilm EC: E. coli gen,0.0,Petrifilm EC: Col Tot,0.0,Mini VIDAS Salmonella spp,0.0,No realizado,NaN
5038,P05039,0.150000,7.966667,8.116667,Canal Bovina >7 °c,No realizado,NaN,No realizado,NaN,Mini VIDAS Salmonella spp,0.0,No realizado,NaN
5043,P05044,0.500000,16.033333,16.533333,Canal Bovina >7 °c,No realizado,NaN,No realizado,NaN,Mini VIDAS Salmonella spp,0.0,No realizado,NaN
5045,P05046,10.433333,16.100000,26.533333,Canal Bovina,No realizado,NaN,No realizado,NaN,Mini VIDAS Salmonella spp,0.0,No realizado,NaN


In [112]:
plot_scatter(
    df=salmo,#[salmo["resultado_salmonella"]>0],
     x_col="horas_toma_a_ingreso",
      y_col="resultado_salmonella",
       group_col="tipo_muestra_producto", 
       size_col="", 
       order_groups=None, 
       hover_data_cols=None, 
       title="", 
       x_title=None, 
       y_title=None, 
       text_format=".1f", 
       marker_size=10, 
       marker_opacity=0.8, 
       marker_colors=None, 
       height=600, 
       width=1000, 
       font_family="Inter, Arial, sans-serif", 
       output_path=None
)

In [113]:
df_good[cls]

,id,horas_toma_a_ingreso,horas_ingreso_a_incubacion,horas_toma_a_incubacion,tipo_muestra_producto,analisis_ecoli_36c_24h,resultado_ec_ufc_cm2,analisis_coliformes_totales_36c_24h,resultado_ct_ufc_cm2,analisis_salmonella_spp_41_5c_24h,resultado_salmonella,analisis_ecoli_o157_h7_41_5c_24h,resultado_ecoli_o157_h7
0,P00001,0.166667,10.000000,10.166667,Canal Porcina,Petrifilm EC: E. coli gen,1.50,Petrifilm EC: Col Tot,2.0,No realizado,NaN,No realizado,NaN
2,P00003,0.216667,106.700000,106.916667,Canal Bovina,No realizado,NaN,No realizado,NaN,No realizado,NaN,mini VIDAS E. coli O157:H7,0.0
4,P00005,0.216667,7.166667,7.383333,Canal Bovina,Petrifilm EC: E. coli gen,0.25,Petrifilm EC: Col Tot,0.0,No realizado,NaN,No realizado,NaN
5,P00006,0.250000,7.416667,7.666667,Canal Porcina,Petrifilm EC: E. coli gen,0.42,Petrifilm EC: Col Tot,1.0,No realizado,NaN,No realizado,NaN
6,P00007,0.166667,6.166667,6.333333,Canal Bovina,Petrifilm EC: E. coli gen,0.08,Petrifilm EC: Col Tot,0.0,No realizado,NaN,No realizado,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8062,P08063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8063,P08064,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8064,P08065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8065,P08066,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
plot_scatter(
    df=df_good[df_good["resultado_ct_ufc_cm2"]>100],
     x_col="horas_ingreso_a_incubacion",
      y_col="resultado_ct_ufc_cm2",
       group_col="tipo_muestra_producto", 
       size_col="", 
       order_groups=None, 
       hover_data_cols=None, 
       title="", 
       x_title=None, 
       y_title=None, 
       text_format=".1f", 
       marker_size=10, 
       marker_opacity=0.8, 
       marker_colors=None, 
       height=600, 
       width=1000, 
       font_family="Inter, Arial, sans-serif", 
       output_path=None
)